In [1]:
# Dependencies
import pandas as pd
import numpy as np
import json
import matplotlib
# from sodapy import Socrata
from config import census_key
import pprint
import requests

In [2]:
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [3]:
cdc_data = pd.read_csv("../Project_1/cdc_data.csv")

cdc_df = pd.DataFrame(cdc_data)
cdc_df.columns
cdc_df = cdc_df[['case_month', 'res_state', 'res_county', 'age_group', 'sex', 'race', 'ethnicity', 'current_status']]
# cdc_df.value_counts('race')
cdc_df.head(100)



,case_month,res_state,res_county,age_group,sex,race,ethnicity,current_status
0,2021-11,MN,STEELE,18 to 49 years,Male,White,Non-Hispanic/Latino,Laboratory-confirmed case
1,2022-01,MN,SCOTT,18 to 49 years,Unknown,Unknown,Missing,Laboratory-confirmed case
2,2021-11,MN,STEARNS,50 to 64 years,Female,White,Non-Hispanic/Latino,Laboratory-confirmed case
3,2020-05,MN,KANDIYOHI,18 to 49 years,Male,White,Non-Hispanic/Latino,Laboratory-confirmed case
4,2021-05,MN,ANOKA,18 to 49 years,Male,NaN,NaN,Laboratory-confirmed case
...,...,...,...,...,...,...,...,...
95,2021-12,MN,CROW WING,50 to 64 years,Female,White,Non-Hispanic/Latino,Probable Case
96,2022-01,MN,RICE,18 to 49 years,Male,Unknown,Missing,Laboratory-confirmed case
97,2020-11,MN,CARVER,18 to 49 years,Male,Unknown,Missing,Laboratory-confirmed case
98,2020-11,MN,SHERBURNE,0 - 17 years,Male,White,Non-Hispanic/Latino,Laboratory-confirmed case


In [22]:
# Census data 2019
url = "https://api.census.gov/data/2019/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)

census_df_2019 = json_to_dataframe(response)

# Split NAME into county & state
name = census_df_2019['NAME'].str.split(",", n=1, expand=True)
COUNTY = census_df_2019['County']=name[0]
STATE = census_df_2019['State']=name[1]
census_df_2019.drop(columns=["NAME"], inplace=True)

# Rename columns to something intelligible
census_df_2019 = census_df_2019[['County', 'State', 'B19013_001E', 'B01003_001E', 'B01002_001E', 'B19301_001E', 'B17001_002E', 'B23025_005E']]
census_df_2019 = census_df_2019.rename(columns={
    'B01003_001E':'Population',
    'B01002_001E': 'Median Age',
    'B19013_001E': 'Household Income',
    'B19301_001E': 'Per Capita Income',
    'B17001_002E': 'Poverty Count',
    'B23025_005E': 'Unemployment Count'
})
# Remove ...County from County in dataframe
census_df_2019['County'] = census_df_2019['County'].replace('County', '', regex=True)

# Upper case for County
census_df_2019['County'] = census_df_2019['County'].str.upper()


census_df_2019['Year'] = 2019
census_df_2019

,County,State,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Year
0,MEEKER,Minnesota,63452,23105,42.2,32368,1524,244,2019
1,RICE,Minnesota,68584,66185,36.5,31221,5301,1356,2019
2,RED LAKE,Minnesota,58576,4015,42.7,30037,416,59,2019
3,CLAY,Minnesota,65269,63446,32.6,30768,7329,977,2019
4,FILLMORE,Minnesota,61207,20949,42.3,30392,2201,285,2019
...,...,...,...,...,...,...,...,...,...
82,LYON,Minnesota,57730,25758,35.8,29634,3473,458,2019
83,MARSHALL,Minnesota,60118,9372,43.7,30746,637,108,2019
84,RENVILLE,Minnesota,59028,14652,43.8,31253,1420,218,2019
85,WASHINGTON,Minnesota,96671,255938,39.5,45822,10822,3461,2019


In [23]:
# Census data 2020
url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)

census_df_2020 = json_to_dataframe(response)

# Split NAME into county & state
name = census_df_2020['NAME'].str.split(",", n=1, expand=True)
COUNTY = census_df_2020['County']=name[0]
STATE = census_df_2020['State']=name[1]
census_df_2020.drop(columns=["NAME"], inplace=True)

# Rename columns to something intelligible
census_df_2020 = census_df_2020[['County', 'State', 'B19013_001E', 'B01003_001E', 'B01002_001E', 'B19301_001E', 'B17001_002E', 'B23025_005E']]
census_df_2020 = census_df_2020.rename(columns={
    'B01003_001E':'Population',
    'B01002_001E': 'Median Age',
    'B19013_001E': 'Household Income',
    'B19301_001E': 'Per Capita Income',
    'B17001_002E': 'Poverty Count',
    'B23025_005E': 'Unemployment Count'
})

# Remove ...County from County in dataframe
census_df_2020['County'] = census_df_2020['County'].replace('County', '', regex=True)

# Upper case for County
census_df_2020['County'] = census_df_2020['County'].str.upper()

census_df_2020['Year'] = 2020
census_df_2020


,County,State,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Year
0,ANOKA,Minnesota,84379,353775,38.6,37804,20270,7236,2020
1,BECKER,Minnesota,60508,34227,42.3,34226,3588,651,2020
2,BELTRAMI,Minnesota,50525,46784,34.1,26563,8167,1390,2020
3,BENTON,Minnesota,60564,40476,36.0,31515,3357,750,2020
4,BIG STONE,Minnesota,55909,4974,48.5,30588,596,64,2020
...,...,...,...,...,...,...,...,...,...
82,RENVILLE,Minnesota,58542,14572,44.0,31243,1373,209,2020
83,ROSEAU,Minnesota,62304,15259,41.6,31452,1133,353,2020
84,SHERBURNE,Minnesota,88671,96015,36.1,36022,4953,1272,2020
85,STEELE,Minnesota,68172,36710,39.2,34648,2887,722,2020
